In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from FinMind.data import DataLoader
api = DataLoader()

In [ ]:
today = datetime.today() #datetime.strptime('2025-04-30', "%Y-%m-%d")
start_date = (today - relativedelta(years=2)).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")
print('start date', start_date)
print('end date', end_date)

ticker = '00830'
df = api.taiwan_stock_daily(
    stock_id=ticker,
    start_date=start_date,
    end_date=end_date
)

In [ ]:
class DCA_With_Dip_Buying:
    """
    定期定額 + 大跌加碼策略
    """
    def __init__(self, base_monthly_investment=10000):
        self.base_monthly = base_monthly_investment
        self.investment_log = []
        self.shares_owned = 0
        self.total_invested = 0

    def get_monthly_fixed_dates(self, start_date, end_date, day_of_month=15):
        """
        生成每月的固定日期（若遇假日，則使用前一個交易日）
        
        Parameters:
        -----------
        start_date : str or datetime
            開始日期
        end_date : str or datetime
            結束日期
        day_of_month : int
            每月的第幾天（1-31）
        """
        # 生成理論上的每月固定日期
        monthly_dates = pd.date_range(
            start=start_date,
            end=end_date,
            freq=f'MS'  # 每月開始
        )
        
        # 將每個月調整到指定日期
        fixed_dates = []
        for date in monthly_dates:
            # 嘗試創建指定日期的日期
            try:
                target_date = date.replace(day=day_of_month)
            except ValueError:
                # 如果該月沒有指定日期（如2月30日），則使用該月最後一天
                last_day = pd.Timestamp(date.year, date.month, 1) + pd.offsets.MonthEnd(1)
                target_date = last_day
            
            fixed_dates.append(target_date)
        
        return pd.DatetimeIndex(fixed_dates)

    def apply_strategy(self, df, invest_day=None):
        """
        應用策略
        """
        if invest_day is None:
            invest_day = self.invest_day

        # 取得每月投資日期
        self.monthly_date = self.get_monthly_fixed_dates(
            start_date=start_date,
            end_date=end_date,
            day_of_month=invest_day)
        
        # 設定加碼條件
        dip_thresholds = {
            'small_dip': 0.03,    # 單日跌3%
            'medium_dip': 0.05,   # 單日跌5%
            'big_dip': 0.08       # 單日跌8%
        }
        
        # 加碼倍數
        dip_multipliers = {
            'small_dip': 1.5,     # 加碼1.5倍
            'medium_dip': 2.0,    # 加碼2倍
            'big_dip': 3.0        # 加碼3倍
        }
        
        
        for i in range(1, len(df)):
            current_date = df['date'].iloc[i]
            current_price = df['close'].iloc[i]
            prev_price = df['close'].iloc[i-1]
        
            # 計算日跌幅
            daily_return = (current_price - prev_price) / prev_price
            
            # 檢查是否是定期定額日
            is_monthly_day = current_date in self.monthly_date
            
            # 決定投資金額
            investment_amount = 0
            
            if is_monthly_day:
                investment_amount = self.base_monthly
                
            # 檢查是否觸發加碼條件
            if daily_return < -dip_thresholds['big_dip']:
                investment_amount += self.base_monthly * dip_multipliers['big_dip']
                trigger = "大跌加碼(>8%)"
            elif daily_return < -dip_thresholds['medium_dip']:
                investment_amount += self.base_monthly * dip_multipliers['medium_dip']
                trigger = "中跌加碼(>5%)"
            elif daily_return < -dip_thresholds['small_dip']:
                investment_amount += self.base_monthly * dip_multipliers['small_dip']
                trigger = "小跌加碼(>3%)"
            elif is_monthly_day:
                trigger = "定期定額"
            else:
                trigger = None
                
            if investment_amount > 0:
                shares_bought = investment_amount / current_price
                self.shares_owned += shares_bought
                self.total_invested += investment_amount
                
                self.investment_log.append({
                    'date': current_date,
                    'price': current_price,
                    'amount': investment_amount,
                    'shares': shares_bought,
                    'trigger': trigger
                })
                
        return pd.DataFrame(self.investment_log)
    
    def get_performance(self, current_price):
        """
        計算績效
        """
        if self.shares_owned == 0:
            return 0, 0
            
        current_value = self.shares_owned * current_price
        total_return = current_value - self.total_invested
        return_rate = (total_return / self.total_invested) * 100
        
        return current_value, total_return, return_rate

In [ ]:
# 執行策略
strategy = DCA_With_Dip_Buying(base_monthly_investment=5000)
log_df = strategy.apply_strategy(df, 9)

current_price = df['close'].iloc[-1]
current_value, total_return, return_rate = strategy.get_performance(current_price)

print(f"總投入金額: {strategy.total_invested:,.0f}元")
print(f"目前持有市值: {current_value:,.0f}元")
print(f"總報酬: {total_return:,.0f}元")
print(f"報酬率: {return_rate:.2f}%")
print(f"持有股數: {strategy.shares_owned:.2f}股")

# 顯示投資記錄
print("\n最近10次投資記錄:")
print(log_df.tail(10).to_string())